<a href="https://www.kaggle.com/misterjjunpatch/titanic-ml-80-accurate-on-test-data?scriptVersionId=87190137" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import matplotlib.pyplot as plt 
from sklearn.impute import SimpleImputer


import seaborn as sns
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


# Getting Data

In [2]:
train_df = pd.read_csv('/kaggle/input/titanic/train.csv')
test_y_df = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')
test_df = pd.read_csv('/kaggle/input/titanic/test.csv')

In [3]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


#### I think ticket is not relevant to predict

In [5]:
train_df = train_df.drop(['PassengerId','Ticket','Name','Cabin'],axis=1)
test_df = test_df.drop(['PassengerId','Ticket','Name','Cabin'],axis=1)

# Data Missing Value

In [6]:
train_df.isnull().any()

Survived    False
Pclass      False
Sex         False
Age          True
SibSp       False
Parch       False
Fare        False
Embarked     True
dtype: bool

# Train-Test Splited

In [7]:
Y_train = train_df.Survived
train_df = train_df.drop("Survived",axis=1)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train_df, Y_train, train_size=0.8, test_size=0.2,random_state=0)

# FindMissing Value

In [9]:
def FindMissingCols(data):
    cols = []
    for col in data.columns:
        if data[col].isnull().any():
            cols.append(col)
    return cols
def PrintMissingforEachColumns(data):
    missing_val_count_by_column = (data.isnull().sum())
    print(missing_val_count_by_column[missing_val_count_by_column > 0])
def PrintUniqueValue(data,cols):
    for col in cols:
        print(data[col].unique())
    

In [10]:
colsTrain = FindMissingCols(X_train)
colsTrain

['Age', 'Embarked']

## Missing Value for each columns

In [11]:
print("Train",X_train.shape)
PrintMissingforEachColumns(train_df)
print("Train",X_valid.shape)
PrintMissingforEachColumns(X_valid)

Train (712, 7)
Age         177
Embarked      2
dtype: int64
Train (179, 7)
Age    36
dtype: int64


# Imputation

In [12]:
def Impute(Train,Test,object,dtype = "obj"):
    Train = Train.copy()
    Test =Test.copy()
    if dtype == 'obj':
        string = 'most_frequent'
    else:
        string = 'mean'
    my_imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    imputed_X_train = pd.DataFrame(my_imputer.fit_transform(Train))
    imputed_X_valid = pd.DataFrame(my_imputer.transform(Test))
    imputed_X_train.columns = Train.columns
    imputed_X_valid.columns = Test.columns
    return imputed_X_train, imputed_X_valid

In [13]:
s = (train_df.dtypes == 'object')
n = (train_df.dtypes != 'object')
object_cols = list(s[s].index)
num_cols = list(n[n].index)
print("Categorical variables:")
# object_cols.remove('Cabin')
print(object_cols)

Categorical variables:
['Sex', 'Embarked']


In [14]:
imputed_X_train,imputed_X_valid = Impute(X_train,X_valid,object_cols,dtype='obj')
imputed_X_train,imputed_X_valid = Impute(imputed_X_train,imputed_X_valid,num_cols,dtype='num')
imputed_X_train.head(),imputed_X_valid.head()

(  Pclass     Sex   Age SibSp Parch     Fare Embarked
 0      3  female  24.0     0     2  15.2458        C
 1      2    male  31.0     0     0     10.5        S
 2      2    male  31.0     1     1  37.0042        C
 3      3    male  20.0     0     0   4.0125        C
 4      3    male  21.0     0     0     7.25        S,
   Pclass     Sex   Age SibSp Parch      Fare Embarked
 0      3    male  24.0     0     0   14.4583        C
 1      3    male  24.0     0     0      7.55        S
 2      3    male   7.0     4     1    29.125        Q
 3      1  female  24.0     1     0  146.5208        C
 4      3  female  29.0     0     2   15.2458        C)

In [15]:
print("Train",imputed_X_train.shape)
PrintMissingforEachColumns(imputed_X_train)
print("Train",imputed_X_valid.shape)
PrintMissingforEachColumns(imputed_X_valid)

Train (712, 7)
Series([], dtype: int64)
Train (179, 7)
Series([], dtype: int64)


# Nominal Conveter preprocessing

In [16]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

def NominalConvert(Train,Test,object_cols):
    Train = Train.copy()
    Test = Test.copy()

    # Apply ordinal encoder to each column with categorical data
    ordinal_encoder = OrdinalEncoder()
    Train[object_cols] = ordinal_encoder.fit_transform(Train[object_cols])
    Test[object_cols] = ordinal_encoder.transform(Test[object_cols])
    return Train, Test

## Nominal Convert 

In [17]:
imputed_X_train, imputed_X_valid = NominalConvert(imputed_X_train,imputed_X_valid,object_cols)

In [18]:
imputed_X_train.head(),imputed_X_valid.head()

(  Pclass  Sex   Age SibSp Parch     Fare  Embarked
 0      3  0.0  24.0     0     2  15.2458       0.0
 1      2  1.0  31.0     0     0     10.5       2.0
 2      2  1.0  31.0     1     1  37.0042       0.0
 3      3  1.0  20.0     0     0   4.0125       0.0
 4      3  1.0  21.0     0     0     7.25       2.0,
   Pclass  Sex   Age SibSp Parch      Fare  Embarked
 0      3  1.0  24.0     0     0   14.4583       0.0
 1      3  1.0  24.0     0     0      7.55       2.0
 2      3  1.0   7.0     4     1    29.125       1.0
 3      1  0.0  24.0     1     0  146.5208       0.0
 4      3  0.0  29.0     0     2   15.2458       0.0)

# Category Data

In [19]:
def CatAge(Train,Test):
    Train = Train.copy() 
    Test = Test.copy()
    data = [Train, Test]
    for dataset in data:
        dataset['Age'] = dataset['Age'].astype(int)
        dataset.loc[ dataset['Age'] <= 11, 'Age'] = 0
        dataset.loc[(dataset['Age'] > 11) & (dataset['Age'] <= 18), 'Age'] = 1
        dataset.loc[(dataset['Age'] > 18) & (dataset['Age'] <= 22), 'Age'] = 2
        dataset.loc[(dataset['Age'] > 22) & (dataset['Age'] <= 27), 'Age'] = 3
        dataset.loc[(dataset['Age'] > 27) & (dataset['Age'] <= 33), 'Age'] = 4
        dataset.loc[(dataset['Age'] > 33) & (dataset['Age'] <= 40), 'Age'] = 5
        dataset.loc[(dataset['Age'] > 40) & (dataset['Age'] <= 66), 'Age'] = 6
        dataset.loc[ dataset['Age'] > 66, 'Age'] = 6
    return data[0],data[1]
def CatFare(Train,Test):
    Train = Train.copy() 
    Test = Test.copy()
    data = [Train, Test]
    for dataset in data:
        dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
        dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
        dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
        dataset.loc[(dataset['Fare'] > 31) & (dataset['Fare'] <= 99), 'Fare']   = 3
        dataset.loc[(dataset['Fare'] > 99) & (dataset['Fare'] <= 250), 'Fare']   = 4
        dataset.loc[ dataset['Fare'] > 250, 'Fare'] = 5
        dataset['Fare'] = dataset['Fare'].astype(int)
    return data[0],data[1]

## Category Age and Fare

In [20]:
imputed_X_train, imputed_X_valid = CatAge(imputed_X_train,imputed_X_valid)
imputed_X_train, imputed_X_valid = CatFare(imputed_X_train,imputed_X_valid)
imputed_X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0.0,3,0,2,2,0.0
1,2,1.0,4,0,0,1,2.0
2,2,1.0,4,1,1,3,0.0
3,3,1.0,2,0,0,0,0.0
4,3,1.0,2,0,0,0,2.0


# Normalization

In [21]:
from sklearn import preprocessing
def normal(Train,Test):
    Train = Train.copy()
    Test = Test.copy()
    min_max_scaler = preprocessing.StandardScaler()
    x_scaled = min_max_scaler.fit_transform(Train)
    Normal_imputed_X_train = pd.DataFrame(x_scaled)
    Normal_imputed_X_train.columns = Train.columns
    x_scaled = min_max_scaler.transform(Test)
    Normal_imputed_X_valid = pd.DataFrame(x_scaled)
    Normal_imputed_X_valid.columns = Test.columns
    return Normal_imputed_X_train, Normal_imputed_X_valid

In [22]:
Normal_imputed_X_train, Normal_imputed_X_valid = normal(imputed_X_train,imputed_X_valid)
imputed_X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0.0,3,0,2,2,0.0
1,2,1.0,4,0,0,1,2.0
2,2,1.0,4,1,1,3,0.0
3,3,1.0,2,0,0,0,0.0
4,3,1.0,2,0,0,0,2.0


# Train, Test Data

In [23]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return accuracy_score(y_valid, preds)

### RandomForest

In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
forest = RandomForestClassifier(100)
forest.fit(Normal_imputed_X_train,y_train)
test_preds = forest.predict(Normal_imputed_X_valid)
accuracy_score(y_valid, test_preds)

0.8212290502793296

### Gaussian NB

In [25]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(Normal_imputed_X_train, y_train)
accuracy_score(y_valid, clf.predict(Normal_imputed_X_valid))

0.8044692737430168

### KNN

In [26]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(19)
classifier.fit(Normal_imputed_X_train, y_train)
accuracy_score(y_valid, classifier.predict(Normal_imputed_X_valid))

0.8100558659217877

### Decision Tree

In [27]:
from sklearn.tree import DecisionTreeClassifier
clf_gini = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=0 )
clf_gini.fit(Normal_imputed_X_train, y_train)
accuracy_score(y_valid, clf_gini.predict(Normal_imputed_X_valid))

0.8379888268156425

### SVC

In [28]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
clf = make_pipeline(StandardScaler(),SVC(gamma='auto',kernel= 'sigmoid'))
clf.fit(Normal_imputed_X_train, y_train)
accuracy_score(y_valid, clf.predict(Normal_imputed_X_valid))

0.7150837988826816

# Test

In [29]:
imputed_X_train,imputed_X_valid = Impute(train_df,test_df,object_cols,dtype='obj')
imputed_X_train,imputed_X_valid = Impute(imputed_X_train,imputed_X_valid,num_cols,dtype='num')
imputed_X_train, imputed_X_valid = NominalConvert(imputed_X_train,imputed_X_valid,object_cols)

imputed_X_train, imputed_X_valid = CatAge(imputed_X_train,imputed_X_valid)
imputed_X_train, imputed_X_valid = CatFare(imputed_X_train,imputed_X_valid)
Normal_imputed_X_train, Normal_imputed_X_valid = normal(imputed_X_train,imputed_X_valid)



In [30]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
clf = make_pipeline(StandardScaler(), SVC(gamma='auto',kernel= 'sigmoid'))
clf.fit(Normal_imputed_X_train, Y_train)
accuracy_score(test_y_df.Survived, clf.predict(Normal_imputed_X_valid)) ,clf.score(Normal_imputed_X_train, Y_train)*100

(0.7918660287081339, 68.5746352413019)

In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(Normal_imputed_X_train, Y_train)

Y_prediction = random_forest.predict(Normal_imputed_X_valid)

random_forest.score(Normal_imputed_X_train, Y_train)
acc_random_forest = round(random_forest.score(Normal_imputed_X_train, Y_train) * 100, 2)
accuracy_score(test_y_df.Survived, Y_prediction),acc_random_forest

(0.861244019138756, 90.35)

In [32]:
rfs_clf = forest_clf = RandomForestClassifier(n_estimators=110, max_depth= 8, max_features='auto',
                                    random_state=0, oob_score=False, min_samples_split = 2,
                                   criterion= 'gini', min_samples_leaf=2, bootstrap=False)
rfs_clf.fit(Normal_imputed_X_train, Y_train)
y_f_predict = rfs_clf.predict(Normal_imputed_X_valid)
acc_random_forest = round(rfs_clf.score(Normal_imputed_X_train, Y_train) * 100, 2)
accuracy_score(test_y_df.Survived, y_f_predict),acc_random_forest

(0.8803827751196173, 87.32)

In [33]:
output = pd.DataFrame({'PassengerId': test_y_df.PassengerId,
                       'Survived':  y_f_predict})
output.to_csv('submission.csv', index=False)
print(output)

     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         1

[418 rows x 2 columns]
